In [1]:
import cloudberry.api as cb

In [2]:
# Define Cloudberry configuration
cb_port = 9000
cb_config = cb.CloudberryConfig(f'http://localhost:{cb_port}')

# Initialize selected API(s)
cb_data = cb.Data(cb_config)
cb_analytics = cb.Analytics(cb_config)
cb_buckets = cb.Buckets(cb_config)
cb_meta = cb.Metadata(cb_config)
cb_query = cb.Query(cb_config)
cb_uploader = cb.AgeFileUploader(cb_config)

In [ ]:
# cb.Data

import math
import datetime

get_current_timestamp = lambda: math.trunc(datetime.datetime.utcnow().timestamp())

current_time = get_current_timestamp()
data_points = [
    cb.DataPoint(time=current_time, fields={'version': 0.1}, tags={'origin': 'jupyter_notebook', 'type': 'api_demo'}),
    cb.DataPoint(time=current_time + 1, fields={'version': 0.2}, tags={'origin': 'jupyter_notebook', 'type': 'api_demo'}),
    cb.DataPoint(time=current_time + 2, fields={'version': 0.1}, tags={'origin': 'notebook', 'type': 'api_demo'}),
    cb.DataPoint(time=current_time + 3, fields={'version': 0.2}, tags={'origin': 'notebook', 'type': 'api_demo'}),
]

# Save data points explicitly
result = cb_data.save_data(data_points)
print(result)

# Get those data points with filters
result = cb_data.get_data(cb.DataFilters(tags={'origin': 'notebook'}))
print(len(result.data))

result = cb_data.get_data(cb.DataFilters(fields={'version': 0.1}))
print(len(result.data))

result = cb_data.get_data(cb.DataFilters(tags={'origin': 'notebook'}, fields={'version': 0.1}))
print(len(result.data), result.data)

# Delete data points
result = cb_data.get_data(cb.DataFilters(tags={'type': 'api_demo'}))
print('before: ', len(result.data))

# result = cb_data.delete_data(cb.DataFilters(tags={'type': 'api_demo'}))
# print(result)

result = cb_data.get_data(cb.DataFilters(tags={'type': 'api_demo'}))
print('after: ', len(result.data))

In [ ]:
# cb.Buckets

buckets_names = cb_buckets.get_buckets_names()
print(buckets_names)

result = cb_buckets.create_bucket('wiadro_0')
print(result)

buckets_names = cb_buckets.get_buckets_names()
print(buckets_names)

result = cb_buckets.delete_bucket('wiadro_0')
print(result)

buckets_names = cb_buckets.get_buckets_names()
print(buckets_names)

In [ ]:
# cb.Query

result = cb_query.query("""
from(bucket:"cloudberry-logs")
|> range(start: -1h)
|> filter(fn: (r) => r.type == "api_demo")
""", raw=True)
print(result)

In [5]:
# cb.AgeFileUploader

age_log_file_path = '../data/emas-20190412T120536.log'
age_log_experiment_name = 'EMAS_AGE'
age_log_headers_keys = {
    "[WH]": "[W]",
    "[SH]": "[S]",
    "[BH]": "[B]"
}

evaluation_id = cb_uploader.upload_file(
    file_name=age_log_file_path,
    experiment_name=age_log_experiment_name,
    details=cb.UploadDetails(headers_keys=age_log_headers_keys)
)
print(evaluation_id)


5f35adc51650d32ea7850834


In [8]:
# cb.Analytics

# Upload more data to Cloudberry
emas_files = [
    "../data/emas-20190411T232808.log",
    "../data/emas-20190411T234810.log",
    "../data/emas-20190412T000813.log",
    "../data/emas-20190412T120536.log",
]
evaluation_ids = []
for emas_file in emas_files:
    evaluation_id = cb_uploader.upload_file(
             file_name=emas_file,
             experiment_name=age_log_experiment_name,
             details=cb.UploadDetails(headers_keys=age_log_headers_keys)
         )
    evaluation_ids.append(evaluation_id)

KeyError: 'seriesName'

In [14]:

best_fitness_field = 'BEST_SOLUTION_SO_FAR'
fitness_field = 'AVERAGE_FITNESS'

series = cb_analytics.compare_evaluations(
    evaluation_ids=evaluation_ids,
    compared_field=fitness_field
)

cb.DataSeriesPlots.compare(series, '_time', fitness_field)

TypeError: list indices must be integers or slices, not str